In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import torchvision.datasets as dsets
import torchvision.transforms as transforms

### Read the data and turn into np array

In [30]:
data = pd.read_csv("drugConsumption_BinaryResponse.csv")
data

,id,Age,Gender,Education,Country,Ethnicity,Neuroticism,Extraversion,Openness,Agreeableness,...,Ecstasy,Heroin,Ketamine,legalh,LSD,Meth,Mushrooms,Nicotine,Semer,VSA
0,1.0,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,3.0,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,1884.0,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,-1.19430,1.74091,1.88511,0.76096,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
1881,1885.0,-0.95197,-0.48246,-0.61113,-0.57009,-0.31685,-0.24649,1.74091,0.58331,0.76096,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
1882,1886.0,-0.07854,0.48246,0.45468,-0.57009,-0.31685,1.13281,-1.37639,-1.27553,-1.77200,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1883,1887.0,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,0.91093,-1.92173,0.29338,-1.62090,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0


In [31]:
data = pd.DataFrame.to_numpy(data)
n,p = data.shape
data.astype("float32")
print(data.dtype)


NameError: name 'float32' is not defined

In [20]:
predictors = data[:,1:13]
predictors.shape

(1885, 12)

In [21]:
response = data[:,13:]
response.shape

(1885, 19)

### Train/Test split

In [22]:
# split training and testing(1/4 for testing)
testn = int(n*0.25)
trainN = n-testn
#print(trainN)
# indices for test set
indtest = np.random.choice(n,testn,replace=False)
#print(len(indtest))
traindata = np.zeros((trainN,p))
testdata = np.zeros((testn,p))
testi = 0
traini = 0
for i in range(n):
    if i in indtest:
        testdata[testi] = data[i,:]
        testi += 1
    else:
        traindata[traini] = data[i,:]
        traini += 1
    #if 
    if traini > trainN:
        print(testi)
        
print(traindata)
print(testdata)


[[ 2.0000e+00 -7.8540e-02 -4.8246e-01 ...  1.0000e+00  0.0000e+00
   0.0000e+00]
 [ 3.0000e+00  4.9788e-01 -4.8246e-01 ...  0.0000e+00  0.0000e+00
   0.0000e+00]
 [ 4.0000e+00 -9.5197e-01  4.8246e-01 ...  0.0000e+00  0.0000e+00
   0.0000e+00]
 ...
 [ 1.8860e+03 -7.8540e-02  4.8246e-01 ...  1.0000e+00  0.0000e+00
   0.0000e+00]
 [ 1.8870e+03 -9.5197e-01  4.8246e-01 ...  1.0000e+00  0.0000e+00
   0.0000e+00]
 [ 1.8880e+03 -9.5197e-01 -4.8246e-01 ...  1.0000e+00  0.0000e+00
   0.0000e+00]]
[[ 1.00000e+00  4.97880e-01  4.82460e-01 ...  0.00000e+00  0.00000e+00
   0.00000e+00]
 [ 8.00000e+00  4.97880e-01 -4.82460e-01 ...  0.00000e+00  0.00000e+00
   0.00000e+00]
 [ 9.00000e+00  4.97880e-01  4.82460e-01 ...  1.00000e+00  0.00000e+00
   0.00000e+00]
 ...
 [ 1.87500e+03 -9.51970e-01 -4.82460e-01 ...  1.00000e+00  0.00000e+00
   1.00000e+00]
 [ 1.87800e+03  2.59171e+00 -4.82460e-01 ...  0.00000e+00  0.00000e+00
   0.00000e+00]
 [ 1.88000e+03 -7.85400e-02 -4.82460e-01 ...  0.00000e+00  0.00000e+

### Make Dataset/loader class

In [23]:
class drugData(Dataset):
    def __init__(self, data):
        self.n = data.shape[0]
        self.x = torch.from_numpy(data[:,1:13])
        #print(self.x.dtype)
        self.y = torch.from_numpy(data[:,13:])
        
        self.x,self.y = self.x.type(torch.DoubleTensor), self.y.type(torch.DoubleTensor)

        #print(self.x.dtype)
        
    def __getitem__(self, index):
        return (self.x[index], self.y[index])
    
    def __len__(self):
        return self.n

In [16]:
trainDataset = drugData(traindata)

torch.float64
torch.float64


In [48]:
trainDataset = drugData(traindata)
print(trainDataset[0][0].shape)
print(trainDataset[0][1].shape)
print(len(traindata))
#print(traindata[0][0].dytpe)
testDataset = drugData(testdata)

torch.Size([12])
torch.Size([19])
1414


In [25]:
trainloader = torch.utils.data.DataLoader(trainDataset,
                                          batch_size=32,
                                          shuffle=True)

testloader = torch.utils.data.DataLoader(testDataset,
                                          batch_size=32,
                                          shuffle=True)

### Model

In [26]:
ind = 12
outd = 19
class Model(nn.Module):

    def __init__(self, hlayer):
        super(Model, self).__init__()
        self.lin1 = nn.Linear(ind, hlayer)
        self.lin2 = nn.Linear(hlayer, outd)
    
    def forward(self, x):
        a1 = torch.relu(self.lin1(x))
        a2 = torch.sigmoid(self.lin2(a1))
        return a2
        

In [32]:
drugnet = Model(32)
drugnet(trainDataset[0][0].float())

tensor([0.5698, 0.4264, 0.4591, 0.5216, 0.4876, 0.5253, 0.4706, 0.4245, 0.5161,
        0.5862, 0.5157, 0.5102, 0.4350, 0.6571, 0.4869, 0.4923, 0.5508, 0.5924,
        0.5167], grad_fn=<SigmoidBackward>)

### Training

In [45]:
def train(model, criterion, optimizer, epochs = 50,p = True, pinterval = 1):
    lossList = []
    for i in range(epochs):
        runningLoss = 0
        for x, y in trainloader:
            optimizer.zero_grad()
            yhat = model(x.float())
            loss = criterion(yhat.float(), y.float())
            loss.backward()
            optimizer.step()
            runningLoss += loss.item()
        if p == True:
            if i % pinterval == 0:
                print('epoch ', i, ' loss: ', str(runningLoss))
        lossList.append(runningLoss)
    return lossList

In [56]:
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(drugnet.parameters(), lr=0.03)
results = train(drugnet,criterion,optimizer,2000,pinterval=200)

epoch  0  loss:  12.679917007684708
epoch  200  loss:  12.593414008617401
epoch  400  loss:  12.459911659359932
epoch  600  loss:  12.536751747131348
epoch  800  loss:  12.511132284998894
epoch  1000  loss:  12.515483796596527
epoch  1200  loss:  12.436059087514877
epoch  1400  loss:  12.321629837155342
epoch  1600  loss:  12.241460904479027
epoch  1800  loss:  12.213866487145424


### Testing Performance

In [57]:
def testAccuracy(data, model,p= True):
    for i in range(len(data)):
        with torch.no_grad():
            yhat = model(data[i][0].float())
        yhat = torch.round(yhat)
        y = data[i][1]
        #print("y", y)
        #print("yhat", yhat)
        return(yhat,y)
        break
yhat, y = testAccuracy(trainDataset,drugnet)
yhat - y

tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -1.,  0.,  0.,  0.,  0.], dtype=torch.float64)

In [ ]:
def testAccuracy(dloader,model,p = False):
    totcount = 0
    correctcount = 0
    for x,y in dloader:
        with torch.no_grad():
            yhat = model(x)
        yhat = torch.round(yhat)
        for i,j in zip(yhat,y):
            if i.item() == j.item():
                correctcount +=1
            totcount += 1
    if p == True:
        print(correctcount)
        print(totcount)
    return correctcount/totcount